In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/"))

# Any results you write to the current directory are saved as output.

In [ ]:
app_train = pd.read_csv("../input/home-credit-default-risk/application_train.csv")
app_train = app_train[app_train["CODE_GENDER"]!='XNA']
app_train = app_train[app_train['NAME_INCOME_TYPE'] != 'Maternity leave']
app_train = app_train[app_train['NAME_FAMILY_STATUS'] != 'Unknown']

# oof_rf = pd.read_csv("../input/rf-script/oof_rf.csv")
# oof_et = pd.read_csv("../input/et-script/oof_et.csv")
oof_lgbm = pd.read_csv("../input/lgbm-script/oof_lgbm.csv")
oof_cb = pd.read_csv("../input/catboost-script/oof_catboost.csv")
oof_xg = pd.read_csv("../input/xgb-result/oof_xgboost.csv")

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

print(roc_auc_score(app_train["TARGET"],oof_lgbm["TARGET"]))
print(roc_auc_score(app_train["TARGET"],oof_xg["TARGET"]))
# print(roc_auc_score(app_train["TARGET"],oof_rf["TARGET"]))
print(roc_auc_score(app_train["TARGET"],oof_cb["TARGET"]))
# print(roc_auc_score(app_train["TARGET"],oof_et["TARGET"]))

In [ ]:
best_score = 0
for i in np.linspace(0,1,21):
    for j in np.linspace(0,1,21):
        x_stack = i*oof_lgbm["TARGET"]+j*oof_xg["TARGET"]+(1-i-j)*oof_cb["TARGET"]
        score = roc_auc_score(app_train["TARGET"],x_stack)
        if score>best_score:
            best_score = score
            print("%.2f,%.2f,%.2f:%f" % (i,j,1-i-j,score))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold

oof_preds = np.zeros(oof_lgbm.shape[0])
x_stack = pd.DataFrame({"x1":oof_lgbm["TARGET"],"x2":oof_xg["TARGET"],"x3":oof_cb["TARGET"]})
folds = KFold(n_splits= 5, shuffle=True, random_state=1001)
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(x_stack, app_train['TARGET'])):
    train_x, train_y = x_stack.iloc[train_idx], app_train['TARGET'].iloc[train_idx]
    valid_x, valid_y = x_stack.iloc[valid_idx], app_train['TARGET'].iloc[valid_idx]
    logreg = LogisticRegression()
    logreg.fit(train_x,train_y)
    oof_preds[valid_idx] = logreg.predict_proba(valid_x)[:,1]
    print(roc_auc_score(valid_y,oof_preds[valid_idx]))
print(roc_auc_score(app_train['TARGET'],oof_preds))

In [ ]:
submission_lgbm = pd.read_csv("../input/lgbm-script/submission_lgbm.csv")
submission_xg = pd.read_csv("../input/xgb-result/submission_xgboost.csv")
submission_cb = pd.read_csv("../input/catboost-script/submission_catboost.csv")
submission_blend = pd.read_csv("../input/homecredit-blend/submission_blend.CSV")

In [ ]:
submission = pd.DataFrame()
submission["SK_ID_CURR"] = submission_lgbm["SK_ID_CURR"]
submission["TARGET"] = 0.65*submission_lgbm["TARGET"]+0.2*submission_xg["TARGET"] + 0.15*submission_cb["TARGET"]
submission.to_csv("submission_blend.csv",index=False)

In [ ]:
logreg = LogisticRegression()
x_stack = pd.DataFrame({"x1":oof_lgbm["TARGET"],"x2":oof_xg["TARGET"],"x3":oof_cb["TARGET"]})
logreg.fit(x_stack,app_train["TARGET"])

submission = pd.DataFrame()
submission["SK_ID_CURR"] = submission_lgbm["SK_ID_CURR"]
x_test = pd.DataFrame({"x1":submission_lgbm["TARGET"],"x2":submission_xg["TARGET"],"x3":submission_cb["TARGET"]})
submission["TARGET"] = logreg.predict_proba(x_test)[:,1]
submission.to_csv("submission_stack.csv",index=False)

In [ ]:
# data = {}
# data["lgbm"] = oof_lgbm
# data["xg"] = oof_xg
# # for path in glob.glob("../input/homecredt/*.csv", recursive=True):
# #     data[path[19:-4]] = pd.read_csv(path)

# ranks = pd.DataFrame(columns=data.keys())
# for key in data.keys():
#     ranks[key] = data[key].TARGET.rank(method='min')
# ranks['Average'] = ranks.mean(axis=1)
# ranks['Scaled Rank'] = (ranks['Average'] - ranks['Average'].min()) / (ranks['Average'].max() - ranks['Average'].min())
# ranks.corr()[:1]

# weights = [0.6,0.4]
# ranks['Score'] = ranks[['lgbm','xg']].mul(weights).sum(1) / ranks.shape[0]
# print(roc_auc_score(app_train["TARGET"],ranks['Score']))